## Setup

In [ ]:
!python -m spacy download en_core_web_sm

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:

!pip install openai==0.27.7

In [ ]:
!pip install -U gradio sentence-transformers rank_bm25

In [ ]:
!pip install -U langchain-groq

In [ ]:
# get important libraries
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
from langchain_groq import ChatGroq
from openai.embeddings_utils import cosine_similarity

## Read the Data For Miami Hotels

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/hamzafarooq/maven-mlsystem-design-cohort-1/main/data/miami_hotels.csv')
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,prices,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place


In [ ]:
df.shape

(2511, 27)

In [ ]:
df.name.value_counts()

,count
name,
Faena Miami Beach,45
1 Hotel South Beach,45
The Kent Hotel,45
The Goodtime Hotel,45
The Betsy South Beach,45
...,...
"The Julia Hotel, Miami Beach",6
Bikini Lodge,4
Beach Park Hotel,4


In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape

(2511, 27)

In [ ]:
df['combined'] = (
    'title: '+ df.title.str.strip()+'; Content: '+ df.review.str.strip()
)

In [ ]:
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,title: My favorite hotel in Miami--Petar is th...
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,title: Exquisite Stay; Content: There are not ...
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,title: Burlesqu Show; Content: I went for the ...
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,title: Dana at Gitano is a rockstar!; Content:...
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,title: Perfect Place; Content: I would like to...


## Preprocessing

In [ ]:
import re

df_combined = df.copy()

df['combined'] = df_combined['combined'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df['combined'] = df['combined'].apply(lambda x: lower_case(x))

In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# get the sample from the data
sample = df_combined[:10]

In [ ]:
sample['embedding'] = sample.combined.apply(lambda x: embedder.encode(x,))

<ipython-input-26-473f0bc372d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['embedding'] = sample.combined.apply(lambda x: embedder.encode(x,))


In [ ]:
sample.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined,embedding
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,title: My favorite hotel in Miami--Petar is th...,"[0.06298833, -0.019645965, -0.03958894, -0.019..."
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,title: Exquisite Stay; Content: There are not ...,"[0.12156845, -0.000459205, 0.005689772, 0.0622..."
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,title: Burlesqu Show; Content: I went for the ...,"[-0.0627091, -0.060777534, -0.043535724, -0.05..."
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,title: Dana at Gitano is a rockstar!; Content:...,"[0.0015523372, -0.042440113, -0.05088378, -0.0..."
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,title: Perfect Place; Content: I would like to...,"[0.061157275, 0.0052637393, -0.02442263, 0.034..."


In [ ]:
query = 'Not worth the effort or money + This hotel is not worth the effort or the price'

query_embedding = embedder.encode(query, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
sample.combined[0]

'title: My favorite hotel in Miami--Petar is the best!; Content: Hands down my absolute favorite hotel in South Beach—there’s no place I’d rather stay. I’ve been back four times in the past six months –each experience is better than the last and I always look forward to coming back. As impressive as the hotel is, the team of people are what make each stay exceptional. Everyone is so friendly and they go out of their way to ensure that I have a perfect stay every single time. Petar in particular always ensures that my stay is wonderful every time and gives me total peace of mind every time I stay there - I never have to worry about a thing and he goes above and beyond to provide excellent service.\n\nI’ve stayed at many hotels in South Beach/Miami and they’re in a league of their own.'

In [ ]:
corpus_embeddings = embedder.encode(sample.combined, show_progress_bar=True)

sample['similarity'] = cosine_similarity(corpus_embeddings, query_embedding)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-30-03178a9ce14c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['similarity'] = cosine_similarity(corpus_embeddings, query_embedding)


In [ ]:
sample

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined,embedding,similarity
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,title: My favorite hotel in Miami--Petar is th...,"[0.06298833, -0.019645965, -0.03958894, -0.019...",0.107516
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,title: Exquisite Stay; Content: There are not ...,"[0.12156845, -0.000459205, 0.005689772, 0.0622...",0.101811
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,title: Burlesqu Show; Content: I went for the ...,"[-0.0627091, -0.060777534, -0.043535724, -0.05...",0.022568
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,title: Dana at Gitano is a rockstar!; Content:...,"[0.0015523372, -0.042440113, -0.05088378, -0.0...",0.063749
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,title: Perfect Place; Content: I would like to...,"[0.061157275, 0.0052637393, -0.02442263, 0.034...",0.076082
5,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I have to create time to come write this revie...,Stay at Faena hotel,title: Stay at Faena hotel; Content: I have to...,"[0.12003811, -0.08685296, -0.035572503, 0.0585...",0.111610
6,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Faena is over-rated. \n\nIt would be a 3 star ...,Overrated. There are better luxury options on ...,title: Overrated. There are better luxury opti...,"[0.07653637, -0.052313346, -0.029117143, 0.019...",0.119061
7,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,The service at the Faena is exceptional. From ...,5 star service ALL around,title: 5 star service ALL around; Content: The...,"[0.003938736, -0.025665278, 0.025575176, 0.020...",0.042064
8,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,My girlfriend and I had the dreamiest stay! Ev...,The perfect l

In [ ]:
# apply at the entire data
df_combined['embedding'] = df_combined.combined.apply(lambda x: embedder.encode(x,))

In [ ]:
df_combined.to_pickle('Entire_data.pkl')

In [ ]:
df_combined.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined,embedding
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,title: My favorite hotel in Miami--Petar is th...,"[0.06298833, -0.019645965, -0.03958894, -0.019..."
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,title: Exquisite Stay; Content: There are not ...,"[0.12156845, -0.000459205, 0.005689772, 0.0622..."
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,title: Burlesqu Show; Content: I went for the ...,"[-0.0627091, -0.060777534, -0.043535724, -0.05..."
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,title: Dana at Gitano is a rockstar!; Content:...,"[0.0015523372, -0.042440113, -0.05088378, -0.0..."
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,title: Perfect Place; Content: I would like to...,"[0.061157275, 0.0052637393, -0.02442263, 0.034..."


In [ ]:
# function to search through reviews for a specific product
def search_reviews(df, query, n = 5, pprint= True):

  #embed the search query
  query_embedding = embedder.encode(query, show_progress_bar=True)

  df['similarity'] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(384, -1)))

  results = df.sort_values('similarity', ascending=False).head(n)

  return results

In [ ]:
query = 'hotel not too expensive and close to beach'

results = search_reviews(df_combined, query, n = 15)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
results

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined,embedding,similarity
2505,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,"This hotel is in a great location, just a shor...",Great Location,title: Great Location; Content: This hotel is ...,"[0.08227934, -0.00840654, 0.0056563, 0.0444561...",[0.79064935]
612,2316201,HOTEL,The Shepley Hotel,https://media-cdn.tripadvisor.com/media/photo-...,[],42,$$$,$193 - $319,hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,http://www.theshepleyhotel.com,#42 of 235 hotels in Miami Beach,235,980,"Comfortable bed, clean room, good housekeeping...","Quiet, small hotel close to the beach","title: Quiet, small hotel close to the beach; ...","[0.08305095, 0.03850815, 0.042337466, 0.077979...",[0.75513923]
970,649833,HOTEL,Kimpton Angler's Hotel,https://media-cdn.tripadvisor.com/media/photo-...,[],35,$$$$,$214 - $402,hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.anglershotelmiami.com/?cm_mmc=Yext...,#35 of 235 hotels in Miami Beach,235,2430,"Ok hotel but nothing more, no help with luggag...",Nothing special but good location for beach.,title: Nothing special but good location for b...,"[0.08625539, 0.033863712, 0.016991265, 0.00344...",[0.73456997]
2223,558250,HOTEL,"MB Hotel, Trademark Collection by Wyndham",https://media-cdn.tripadvisor.com/media/photo-...,[],157,$$$,$165 - $265,hotel,3.0,...,https://www.tripadvisor.com/Hotel_Review-g3443...,http://www.wyndhamhotels.com/hotel/48182,#157 of 235 hotels in Miami Beach,235,628,Small hotel on the beach clean rooms and great...,Clean and friendly,title: Clean and friendly; Content: Small hote...,"[0.057292007, 0.05112282, 0.05429346, 0.019103...",[0.7204395]
1980,3937200,HOTEL,Pestana Miami South Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],21,$$$,$189 - $321,hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,http://www.pestana.com/en/hotel/pestana-south-...,#21 of 235 hotels in Miami Beach,235,3624,Very nice hotel close by the beach. Big and cl...,Nice staying!,title: Nice staying!; Content: Very nice hotel...,"[0.061206125, 0.032410204, 0.03873944, 0.03884...",[0.715029]
1960,1699819,HOTEL,Dream South Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],22,$$$,$169 - $287,hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.dreamhotels.com/south-beach,#22 of 235 hotels in Miami Beach,235,3271,"Staff are great, friendly and fun, room was go...",Great hotel but….EXPENSIVE,title: Great hotel but….EXPENSIVE; Content: St...,"[0.07510772, -0.030752098, -0.03287225, 0.0488...",[0.7121659]
248,15307694,HOTEL,Hampton Inn Miami Beach - Mid Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],4,$$$,$180 - $318,hotel,5.0,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.hilton.com/en/hotels/miacahx-hampt...,#4 of 235 hotels in Miami Beach,235,792,"Great service, clean room, easy check in and o...",Close to the beach,title: Close to the beach; Content: Great serv...,"[0.05304162, 0.03373307, -0.007626571, 0.00467...",[0.7105477]
1222,224009,HOTEL,Best Western Plus Atlantic Beach Resort,https://media-cdn.tripadvisor.com/media/photo-...,[],151,$$$,$167 - $267,hotel,3.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.bestwestern.com/de_DE/book/hotel-r...,#151 of 235 hotels in Miami Beach,235,2793,Excellent visit - amenities and property are w...,Enjoyable Labor Day Getaway,title: Enjoyable Labor Day Getaway; Content: E...,"[0.07216452, 0.049685255, 0.00546592, 0.055286...",[0.7084735]
899,6959850,HOTEL,The Confidante Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],52,$$$$,$247 - $420,hotel,4.5,...,ht

In [ ]:
# take all reviews which are closest to the query and groupby the hotel name
def search(query):

  n = 15

  query_embedding = embedder.encode(query, show_progress_bar=True)

  df_combined['similarity'] = df_combined.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(384, -1)))

  results = df_combined.sort_values('similarity', ascending=False).head(n)

  result_list = []

  # make them in very concise and orderd manner
  hlist = []

  for i in results.index:
    if results.name[i] not in hlist:
      small_df = results.loc[results.name == results.name[i]]
      if small_df.shape[1] > 3:
        small_df = small_df[:3]

      result_list.append({
          'name': results.name[i],
          'score': small_df.similarity[i][0],
          'rating': small_df.rating.max(),
          'relevant_review': [small_df.review[j] for j in small_df.index]
      })
      hlist.append(results.name[i])


  return result_list

In [ ]:
search('Hotels with ocean view and free breakfast')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[{'name': 'The Betsy South Beach',
  'score': np.float32(0.69484603),
  'rating': 4.5,
  'relevant_review': ["Absolutely wonderful stay. Easily the classiest hotel on Ocean Dr, amazing setting, courteous staff, fantastic breakfasts on the terrace, cocktails can't be faulted (although horrifically expensive), nice pool area (quite exposed when the wind gets up) and spotlessly clean. Overall I recommend if you've deep pockets."]},
 {'name': 'Best Western Plus Atlantic Beach Resort',
  'score': np.float32(0.6931626),
  'rating': 3.5,
  'relevant_review': ['Excellent visit - amenities and property are well-maintained; the hotel staff provided excellent  customer service.  I highly recommend this resort.  The room was very clean and comfortable It  is located very close to the beach which is easily accessed from the pool area.  The pool area provided guests with towels, lounge chairs and a comfortable area to sit and relax after a swim.  We also enjoyed breakfast (NOT included in your stay)

In [ ]:
groq_api_key = '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'

In [ ]:
def generate_answer(query):
    prompt = f"""
    Based on the following query from a user, please generate a detailed answer based on the context
    focusing on which is the top three hotel based on the query. You should respond as you are a travel agent and are conversing with the
    user in a nice cordial way. Always address the user as Travis. Make sure you infrom the user why this is a good answer. The answer should be a paragph
    remove the special characters and (/n ) , make the output clean and concise.
    Answer only as a poet.
    Don't remind the score in the response and remind only the rating
    Don't make it as a structure of sending an email, make it perfect with the user experience

    ###########
    query:
    "{query}"

    ########

    context:"
    "{search(query)}"
    #####

    Return in Markdown format with each hotel highlighted.
    """

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    response = ChatGroq(groq_api_key = groq_api_key, model_name = 'Llama3-8b-8192', streaming = True)

    response = response.invoke(messages)

    # Extract the generated response from the API response
    generated_text = response.content.strip()

    return generated_text

In [ ]:
text = generate_answer('Hotels with ocean view and free breakfast ')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import markdown
from IPython.display import display, HTML

def render_markdown(md_text):
    # Convert Markdown to HTML
    html = markdown.markdown(md_text)
    # Display the HTML
    display(HTML(html))

In [ ]:
render_markdown(text)

In [ ]:
import gradio as gr

def greet(query):
    bm25 = generate_answer(query)
    return bm25

# Use the gradio library to display a user interface for your user to interact with.
demo = gr.Interface(fn=greet, inputs="text", outputs="text")

# Launch the user demo - This can be done directly in your colab notebook. On your local notebook, you can also give a personalized localhost:port address.
demo.launch(share=True,debug=False)
# COPY THE URL THAT APPEARS BELOW

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d226d3775ae4b35c9a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
